In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

from IS2ML_utils import *

In [ ]:
# get the stats for all lakes
fn_lakestats = '../GLD3_GrIS_lakestats.csv'
df = pd.read_csv(fn_lakestats)
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='EPSG:3413')
gdfg = gdf[gdf.label=='good_lake']

In [ ]:
# specify where the data lives (this is from the FLUID-SuRRF output file structure)
# need to adjust the base_dir
kwargs = {
    'base_dir': '/Users/parndt/jupyterprojects/GLD3_complete/GrIS/',  # base directory where the fluid surrf data lives
    'data_dir': 'detection_out_data/',                                # the directory with the data files (structure from output)
    'ground_track_buffer': 7.5,          # radius in meters, for a 11-m diameter footprint and some allowance for geolocation uncertainty
    'max_cloudiness': 20,                # 20 seems to be good, sometimes over lakes the cloudScore thinks there are clouds
    'days_buffer': 7,                    # limit to +/- one week from ICESat-2 acquisition, can later filter for NDWI overlap / correlation with band reflectances
    'min_sun_elevation': 20,             # 20 is commonly used, beelow that data often has issues
    'limit_n_imgs': 20,                  # limit the total number of images queried, ground track points with missing data after this are discarded
    'ndwi_thresh': 0.15,                 # this threshold is used to calculate NDWI overlap between ICESat-2 and Sentinel-2 water extent
    're_calculate_if_existing': False, # re-do calculations even if file already exists
    'FLUID_SuRRF_info': gdfg
}

# for lake_id in gdfg.lake_id[:1]:
#     get_data_and_plot(lake_id=lake_id, **kwargs)

Only uncomment and run the code below when doing a full new run! 

In [ ]:
############################################################################################################
#### for a clean, new run: clear the output data (except for metadata files from Sentinel-2)
############################################################################################################
# folders_to_reset = ['training_data_CSVs', 'imagery_gt', 'atl03_segments']
# for folder in folders_to_reset:
#     ! rm -rf $folder
#     os.makedirs(folder)

In [ ]:
for ilk, lake_id in enumerate(gdfg.lake_id[:2000]):
    print('\n_________________________________________________ lake %5i / %5i _________________________________________________\n' % (ilk+1, len(gdfg)))
    try:
        fig_path, lk, gdf_final, lk_info = get_data_and_plot(lake_id=lake_id, **kwargs)
    except:
        warnings.warn('Getting data failed for lake %i! (%s)' % (ilk+1, lake_id))
        traceback.print_exc()